# CSCI 4253 / 5253 - Lab #4 - Patent Problem with Spark DataFrames
<div>
 <h2> CSCI 4283 / 5253 
  <IMG SRC="https://www.colorado.edu/cs/profiles/express/themes/cuspirit/logo.png" WIDTH=50 ALIGN="right"/> </h2>
</div>

This [Spark cheatsheet](https://s3.amazonaws.com/assets.datacamp.com/blog_assets/PySpark_SQL_Cheat_Sheet_Python.pdf) is useful as is [this reference on doing joins in Spark dataframe](http://www.learnbymarketing.com/1100/pyspark-joins-by-example/).

The [DataBricks company has one of the better reference manuals for PySpark](https://docs.databricks.com/spark/latest/dataframes-datasets/index.html) -- they show you how to perform numerous common data operations such as joins, aggregation operations following `groupBy` and the like.

In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

The following aggregation functions may be useful -- [these can be used to aggregate results of `groupby` operations](https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-python.html#example-aggregations-using-agg-and-countdistinct). More documentation is at the [PySpark SQL Functions manual](https://spark.apache.org/docs/2.3.0/api/python/pyspark.sql.html#module-pyspark.sql.functions). Feel free to use other functions from that library.

In [2]:
from pyspark.sql.functions import col, count, countDistinct

Create our session as described in the tutorials

In [3]:
spark = SparkSession \
    .builder \
    .appName("Lab4-Dataframe") \
    .master("local[*]")\
    .getOrCreate()

Read in the citations and patents data and check that the data makes sense. Note that unlike in the RDD solution, the data is automatically inferred to be Integer() types.

In [4]:
citations = spark.read.load('cite75_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [5]:
citations.show(5)

+-------+-------+
| CITING|  CITED|
+-------+-------+
|3858241| 956203|
|3858241|1324234|
|3858241|3398406|
|3858241|3557384|
|3858241|3634889|
+-------+-------+
only showing top 5 rows



In [6]:
patents = spark.read.load('apat63_99.txt.gz',
            format="csv", sep=",", header=True,
            compression="gzip",
            inferSchema="true")

In [7]:
patents.show(5)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+
|3070801| 1963| 1096|   null|     BE|   null|    null|      1|  null|   269|  6|    69| null|       1|    null|    0.0|    null|    null|    null|    null|    null|    null|    null|
|3070802| 1963| 1096|   null|     US|     TX|    null|      1|  null|     2|  6|    63| null|       0|    null|   null|    null|    null|    null|    null|    null|    null|    null|
|3070803| 1963| 1096|   null|     US|     IL|    null|      1|  null|     2|  6|    6

In [8]:
union_input = citations.join(patents, citations["CITING"] == patents["PATENT"], how="left")


In [9]:
union_input = union_input.select("CITED", "CITING", col("POSTATE").alias("CITING_STATE"))
union_input.show()

+-------+-------+------------+
|  CITED| CITING|CITING_STATE|
+-------+-------+------------+
|1331793|3858258|          CA|
|1540798|3858258|          CA|
| 924225|3858527|        null|
|2444326|3858527|        null|
|2705120|3858527|        null|
|2967080|3858527|        null|
|3602157|3858527|        null|
|3638586|3858527|        null|
|3699902|3858527|        null|
| 957631|3858560|          IN|
|3675252|3858597|          MT|
|3815160|3858597|          MT|
|2290722|3858770|          CA|
|2777621|3858770|          CA|
|2782969|3858770|          CA|
|3040941|3858770|          CA|
| 982044|3859029|          NY|
|1020004|3859029|          NY|
|1830227|3859029|          NY|
|2752631|3859029|          NY|
+-------+-------+------------+
only showing top 20 rows



In [10]:
intermediate_output = union_input.join(patents, union_input["CITED"] == patents["PATENT"], how="left")

In [11]:
intermediate_output = intermediate_output.select("CITING", "CITING_STATE", "CITED", col("POSTATE").alias("CITED_STATE"))
intermediate_output.show()

+-------+------------+-----+-----------+
| CITING|CITING_STATE|CITED|CITED_STATE|
+-------+------------+-----+-----------+
|4305315|          MN| 2366|       null|
|4192521|        null| 2366|       null|
|4253355|          MN| 2366|       null|
|5580635|          WI| 5156|       null|
|4976561|        null| 5518|       null|
|4480374|          MN| 5803|       null|
|5123817|        null| 6620|       null|
|4115020|        null| 7240|       null|
|4727698|          CA| 7253|       null|
|4360982|          IA| 7340|       null|
|4108250|          IL| 7340|       null|
|5692807|          PA|10817|       null|
|5581904|        null|11458|       null|
|4282613|          MI|12940|       null|
|4741426|          NY|13840|       null|
|4705153|          NY|13840|       null|
|4556218|          FL|14832|       null|
|4896714|        null|15447|       null|
|5065652|          OH|15790|       null|
|5058476|          OH|15790|       null|
+-------+------------+-----+-----------+
only showing top

In [23]:
intermediate_output = intermediate_output.filter("CITED_STATE is not null")

In [24]:
intermediate_output.filter(col("CITING_STATE") == col("CITED_STATE"))

DataFrame[CITING: int, CITING_STATE: string, CITED: int, CITED_STATE: string]

In [25]:
citing_with_count = intermediate_output.groupby("CITING").count()

In [26]:
citing_with_count.show()

+-------+-----+
| CITING|count|
+-------+-----+
|3858527|    3|
|3858597|    2|
|3859029|    2|
|3859627|    5|
|3860038|    3|
|3860100|    3|
|3860191|    2|
|3860806|    1|
|3860969|    1|
|3861180|    2|
|3861473|    2|
|3861674|    4|
|3862577|    1|
|3862844|    4|
|3862904|    5|
|3863090|    3|
|3863692|    2|
|3863750|    1|
|3863935|    2|
|3863980|    2|
+-------+-----+
only showing top 20 rows



In [28]:
final_output = patents.join(citing_with_count, patents["PATENT"] == citing_with_count["CITING"])

In [29]:
final_output = final_output.drop("CITING")

In [30]:
final_output.orderBy(col("count"), ascending=False).show(10)

+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
| PATENT|GYEAR|GDATE|APPYEAR|COUNTRY|POSTATE|ASSIGNEE|ASSCODE|CLAIMS|NCLASS|CAT|SUBCAT|CMADE|CRECEIVE|RATIOCIT|GENERAL|ORIGINAL|FWDAPLAG|BCKGTLAG|SELFCTUB|SELFCTLB|SECDUPBD|SECDLWBD|count|
+-------+-----+-----+-------+-------+-------+--------+-------+------+------+---+------+-----+--------+--------+-------+--------+--------+--------+--------+--------+--------+--------+-----+
|5887243| 1999|14326|   1995|     US|     NY|  752434|      2|  null|   455|  2|    21|  745|       0|  0.9785|   null|  0.8195|    null| 16.9289|     0.0|     0.0|    null|    null|  524|
|5795784| 1998|14109|   1996|     US|     TX|     735|      2|     3|   436|  1|    19|  770|       0|   0.987|   null|  0.8489|    null| 13.0312|  0.0274|  0.0234|    null|    null|  495|
|5856194| 1999|14249|   1996|     US|     TX|     735| 